In [1]:
import numpy as np
import tensorflow as tf
from tensorflow import keras

import socketio
import asyncio

from flask import Flask
from flask_socketio import SocketIO, send, emit

In [2]:
sio = socketio.Client()

app = Flask(__name__)
app.config['SECRET_KEY'] = 'secret!'
server = SocketIO(app, logger=True, engineio_logger=True)

Server initialized for threading.


In [3]:
@server.on('data', namespace="/recv_data")
def got_data(data):
    print("Received data in part1")
    inpt = tf.convert_to_tensor(data)
    out = model(inpt).numpy().tolist()
    sio.emit('data', data=out, namespace="/recv_data")

@server.on('connect', namespace="/recv_model")
def connect(auth):
    print("Dispatcher Connected, auth:", auth["name"])

@server.on('disconnect', namespace="/recv_model")
def disconnect():
    print("Dispatcher disconnected")

@server.on('connect', namespace="/recv_data")
def connect():
    print("Previous node connected")

@server.on('dispatch', namespace="/recv_model")
def recv_model(model_json, next_node):
    global model
    print(model_json)
    model = tf.keras.models.model_from_json(model_json)
    #model.save("models/part1")
    print(next_node)
    sio.connect(next_node, namespaces='/recv_data')
    print(sio.connected)

In [4]:
if __name__ == '__main__':
    server.run(app, port=5001, debug=True, use_reloader=False)

 * Serving Flask app '__main__' (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: on


 * Running on http://127.0.0.1:5001/ (Press CTRL+C to quit)
vMczZ05sacIllQWjAAAA: Sending packet OPEN data {'sid': 'vMczZ05sacIllQWjAAAA', 'upgrades': ['websocket'], 'pingTimeout': 20000, 'pingInterval': 25000}
127.0.0.1 - - [26/Sep/2021 23:58:48] "GET /socket.io/?transport=polling&EIO=4&t=1632725928.154549 HTTP/1.1" 200 -
vMczZ05sacIllQWjAAAA: Received request to upgrade to websocket
vMczZ05sacIllQWjAAAA: Upgrade to websocket successful
vMczZ05sacIllQWjAAAA: Received packet MESSAGE data 0/recv_model,{"name":"dispatcher"}
vMczZ05sacIllQWjAAAA: Sending packet MESSAGE data 0/recv_model,{"sid":"BzTVokXOrQlRWsqRAAAB"}
vMczZ05sacIllQWjAAAA: Received packet MESSAGE data 2/recv_model,["dispatch","{\"class_name\": \"Functional\", \"config\": {\"name\": \"model\", \"layers\": [{\"class_name\": \"InputLayer\", \"config\": {\"batch_input_shape\": [3, 2], \"dtype\": \"int32\", \"sparse\": false, \"ragged\": false, \"name\": \"input_1\"}, \"name\": \"input_1\", \"inbound_nodes\": []}, {\"class_name

Dispatcher Connected, auth: dispatcher
{"class_name": "Functional", "config": {"name": "model", "layers": [{"class_name": "InputLayer", "config": {"batch_input_shape": [3, 2], "dtype": "int32", "sparse": false, "ragged": false, "name": "input_1"}, "name": "input_1", "inbound_nodes": []}, {"class_name": "Dense", "config": {"name": "dense", "trainable": true, "dtype": "float32", "units": 3, "activation": "linear", "use_bias": true, "kernel_initializer": {"class_name": "Constant", "config": {"value": 1.0}}, "bias_initializer": {"class_name": "Zeros", "config": {}}, "kernel_regularizer": null, "bias_regularizer": null, "activity_regularizer": null, "kernel_constraint": null, "bias_constraint": null}, "name": "dense", "inbound_nodes": [[["input_1", 0, 0, {}]]]}], "input_layers": [["input_1", 0, 0]], "output_layers": [["dense", 0, 0]]}, "keras_version": "2.6.0", "backend": "tensorflow"}
http://127.0.0.1:5002/
True


huNLflh9BmomUPrJAAAC: Sending packet OPEN data {'sid': 'huNLflh9BmomUPrJAAAC', 'upgrades': ['websocket'], 'pingTimeout': 20000, 'pingInterval': 25000}
127.0.0.1 - - [26/Sep/2021 23:58:50] "GET /socket.io/?transport=polling&EIO=4&t=1632725930.7327523 HTTP/1.1" 200 -
huNLflh9BmomUPrJAAAC: Received request to upgrade to websocket
huNLflh9BmomUPrJAAAC: Upgrade to websocket successful
huNLflh9BmomUPrJAAAC: Received packet MESSAGE data 0/recv_data,
huNLflh9BmomUPrJAAAC: Sending packet MESSAGE data 0/recv_data,{"sid":"Jv_Fle3t-fIR1422AAAD"}
huNLflh9BmomUPrJAAAC: Received packet MESSAGE data 2/recv_data,["data",[[1,2],[3,4],[5,6]]]
received event "data" from Jv_Fle3t-fIR1422AAAD [/recv_data]


Previous node connected
Received data in part1


vMczZ05sacIllQWjAAAA: Sending packet PING data None
huNLflh9BmomUPrJAAAC: Sending packet PING data None
huNLflh9BmomUPrJAAAC: Received packet PONG data 
vMczZ05sacIllQWjAAAA: Unexpected error "", closing connection
huNLflh9BmomUPrJAAAC: Unexpected error "", closing connection
127.0.0.1 - - [26/Sep/2021 23:59:35] "GET /socket.io/?transport=websocket&EIO=4&sid=huNLflh9BmomUPrJAAAC&t=1632725930.7577553 HTTP/1.1" 200 -
127.0.0.1 - - [26/Sep/2021 23:59:35] "GET /socket.io/?transport=websocket&EIO=4&sid=vMczZ05sacIllQWjAAAA&t=1632725928.1705472 HTTP/1.1" 200 -


Dispatcher disconnected
